In [1]:
import time
start = time.time()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1643880467653_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import time
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import explode
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col,row_number

# l = [10,18,20]
# df.filter(F.col("score").isin(l))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
file_path = "s3://aws-glue-neptune-data/postgre-data-lake/user_followings_list_v2/partition=2022-02-03/"
# user_followings_list = spark.read.format('parquet').\
# load(file_path).select('user_id', 'following_ids')
user_followings_list = spark.read.format('parquet').load(file_path).\
select('user_id', 'following_ids','following_count')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#following count distribution
# 99 percentile is 307
# 99.9 percentile is 2047

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
user_followings_list = user_followings_list.filter("following_count>0 AND following_count<=300").\
select('user_id', 'following_ids')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
user_followings_list.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- following_ids: array (nullable = true)
 |    |-- element: integer (containsNull = true)

In [8]:
file_path_users= "s3://ml-test-analytics/test/stg_rfy/consideration_set/cs_users_20220203_173818.csv"
cs_users_df = spark.read.csv(
    file_path_users, 
    header=True)
cs_users_df = cs_users_df.select('user_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# file_path_users= "s3://ml-test-analytics/test/stg_rfy/consideration_set/Selected_Users.csv"
# cs_users_df = spark.read.csv(
#     file_path_users, 
#     header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
cs_users_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: string (nullable = true)

In [11]:
cs_users_df = cs_users_df.withColumnRenamed("user_id","cs_user_id")
cs_users_df = cs_users_df.withColumn("cs_user_id",cs_users_df["cs_user_id"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# For testing with a smaller subset 
#cs_users_df = cs_users_df.limit(800)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# # Create hard coded row
# unknown_list = [[52]]
# # turn row into dataframe
# unknown_df = spark.createDataFrame(unknown_list)
# # union with existing dataframe
# cs_users_df = cs_users_df.union(unknown_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#cs_users_df.filter("cs_user_id==52").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
cs_users_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)

In [16]:
#cs_users_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
user_followings_list.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- following_ids: array (nullable = true)
 |    |-- element: integer (containsNull = true)

### Following list for Consideration set

In [18]:
# Consideration set - user following list
cs_ufl = cs_users_df.join(user_followings_list, 
                               cs_users_df.cs_user_id == user_followings_list.user_id,
                               "inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
cs_ufl = cs_ufl.drop(cs_ufl.user_id)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
cs_ufl.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- following_ids: array (nullable = true)
 |    |-- element: integer (containsNull = true)

In [21]:
#cs_ufl.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Exploding Once 

In [22]:
cs_ufl_exp = cs_ufl.select(cs_ufl.cs_user_id,explode(cs_ufl.following_ids).alias('cs_following_user_id'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
cs_ufl_exp.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- cs_following_user_id: integer (nullable = true)

In [24]:
user_followings_list.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- following_ids: array (nullable = true)
 |    |-- element: integer (containsNull = true)

### Getting follows of follows

In [25]:
l1_following_list = cs_ufl_exp.join(user_followings_list, 
                                    cs_ufl_exp.cs_following_user_id == user_followings_list.user_id,
                                    "inner")
l1_following_list = l1_following_list.drop(l1_following_list.user_id)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
l1_following_list.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- cs_following_user_id: integer (nullable = true)
 |-- following_ids: array (nullable = true)
 |    |-- element: integer (containsNull = true)

### Exploding again

In [27]:
l1_ufl_exp = l1_following_list.select(l1_following_list.cs_user_id,
                                      l1_following_list.cs_following_user_id, 
                                      explode(l1_following_list.following_ids).alias('l1_following_user_id'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
l1_ufl_exp.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- cs_following_user_id: integer (nullable = true)
 |-- l1_following_user_id: integer (nullable = true)

### Filtering out invalid rows

In [29]:
cond1 = (l1_ufl_exp.cs_user_id!=l1_ufl_exp.cs_following_user_id) # not following yourself
cond2 = (l1_ufl_exp.cs_following_user_id!=l1_ufl_exp.l1_following_user_id) # cs_following not following themselves
cond3 = (l1_ufl_exp.cs_user_id!=l1_ufl_exp.l1_following_user_id) # not suggesting oneself

l1_ufl_exp = l1_ufl_exp.filter(cond1 & cond2 & cond3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#l1_ufl_exp.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
l1_ufl_exp.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- cs_following_user_id: integer (nullable = true)
 |-- l1_following_user_id: integer (nullable = true)

In [32]:
final_counts_df = l1_ufl_exp.groupBy("cs_user_id", "l1_following_user_id").count()\
.withColumnRenamed("count","fof_score").sort(desc("fof_score"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
final_counts_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- l1_following_user_id: integer (nullable = true)
 |-- fof_score: long (nullable = false)

In [34]:
# final_counts_df.show(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
#final_counts_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Filtering out already following

In [36]:
## To-do
# DF1.except(DF2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
cs_ufl_exp.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- cs_following_user_id: integer (nullable = true)

In [38]:
final_counts_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- l1_following_user_id: integer (nullable = true)
 |-- fof_score: long (nullable = false)

In [39]:
# When you join two DataFrame using Left Anti Join (leftanti), 
#it returns only columns from the left DataFrame for non-matched records.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
fof_potential_df = final_counts_df.join(cs_ufl_exp,
                                        (final_counts_df.cs_user_id == cs_ufl_exp.cs_user_id) &
                                        (final_counts_df.l1_following_user_id == cs_ufl_exp.cs_following_user_id),
                                        how='left_anti')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
#fof_potential_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# Writing to s3
fof_potential_df.write.parquet("s3://ml-test-analytics/test/stg_rfy/stg_user_followings/dryrun/",mode="overwrite")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
fof_potential_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cs_user_id: integer (nullable = true)
 |-- l1_following_user_id: integer (nullable = true)
 |-- fof_score: long (nullable = false)

In [44]:
window = Window.partitionBy(fof_potential_df['cs_user_id']).orderBy(fof_potential_df['fof_score'].desc())

# rank is not consecutive in this case, use row number instead
#fof_potential_df.select('*', rank().over(window).alias('rank')).show() 
  #.filter(col('rank') <= 2) 
  #.show() 

fof_ranked= fof_potential_df.select('*', row_number().over(window).alias('rank')).\
  filter(col('rank') <= 100).select('cs_user_id', 'l1_following_user_id','fof_score')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
fof_ranked.write.parquet("s3://ml-test-analytics/test/stg_rfy/stg_user_followings/dryrun-top100/",mode="overwrite")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
end = time.time()
print(end - start)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

94.04838228225708

In [47]:
#file_path_1 = 's3://ml-test-analytics/test/stg_rfy/stg_user_followings/partition=2022-01-29/100_users_wo_filter/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
# from pyspark.sql.functions import col

# df_wo_filter = spark.read.parquet(file_path_1)
# df_wo_filter = df_wo_filter.select(col("cs_user_id").alias("user_id"), col("l1_following_user_id").alias("l1_user_id"),
#                                    col("fof_score").alias("ff_score"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
# df_wo_filter.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
# file_path_2 = 's3://ml-test-analytics/test/stg_rfy/stg_user_followings/partition=2022-01-29/100_users/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
# df_w_filter = spark.read.parquet(file_path_2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
# df_wo_filter.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
# df_w_filter.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
# fof_test = df_wo_filter.join(df_w_filter ,
#                                         (df_wo_filter.user_id == df_w_filter.cs_user_id) &
#                                         (df_wo_filter.l1_user_id == df_w_filter.l1_following_user_id),
#                                         how='left_anti')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# # therse many records were not found in the with filter
# fof_test.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
# fof_test = df_wo_filter.join(df_w_filter ,
#                                         (df_wo_filter.user_id == df_w_filter.cs_user_id) &
#                                         (df_wo_filter.l1_user_id == df_w_filter.l1_following_user_id),
#                                         how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
# df_filtered= fof_test.filter("cs_user_id IS NOT NULL")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
#df_filtered['bool'] = df_filtered['ff_score'] - df_filtered['fof_score'] 
# df_filtered = df_filtered.withColumn('diff', (df_filtered['ff_score'] - df_filtered['fof_score']))
# df_filtered.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
# # Total connections without filter
# import pyspark.sql.functions as F     

# df_filtered.agg(F.sum("ff_score")).collect()[0][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
# # delta after adding filter
# df_filtered.agg(F.sum("diff")).collect()[0][0]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
# connections we lost
# fof_test.filter("cs_user_id IS NULL").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
#(96594561/97734449) *100.0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
# Records where fof score didn't match
#fof_test.filter("ff_score!=fof_score").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
#fof_test.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
# 0.88%  records didn't have the same fof_score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
#(868555/97734449)*100

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
#fof_potential_df = final_counts_df.select('cs_user_id', 'l1_following_user_id').except(cs_ufl_exp)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
## You should be left with all the records we got from the explodes and count, 
## except the rows that already exist as part of the original cs_user follows
## Inner join this back with final_counts_df to get subset of final_counts_df which has all potential fof connects

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
#fof_potential_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
#fof_potential_df.filter("cs_user_id==52").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
# Writing to s3
# df_fof_res.write.parquet("s3://ml-test-analytics/test/stg_rfy/stg_user_followings/partition=2022-01-29/",mode="overwrite")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…